In [1]:
import sys
import os
sys.path.insert(0, '/opt/research/EchoHiding/')
from echohiding import echo_hide, extract_echo_bits, get_mp3_encoded
import pathlib
import librosa
import numpy
from util_functions import *

In [2]:
L = 2048
avgs = []

directory = '/opt/research/fma_medium/'

In [5]:
# Insert all 0's into the audio clips

y, sr = librosa.load("/opt/research/fma_medium/000/000002.mp3")

bitrate = 64
b_est_mp3 = extract_echo_bits(get_mp3_encoded(y, sr, bitrate), L)

zeros = numpy.ones(len(b_est_mp3))
z = echo_hide(y, L, zeros, alpha=0.2)

b_est_mp3 = extract_echo_bits(get_mp3_encoded(z, sr, bitrate), L)
print(numpy.mean(b_est_mp3))

1.0


In [ ]:
# Calc averages of each subdirectory of audio clips and save to a flat file

results = []
# using with statement
with open('natural_avgs.txt', 'a') as file:
    for i in range(156):
        avg = process_dir("{}{}".format(directory, "{0:03}".format(i)), get_avg_echo_vals)
        print("Directory: {} | Average: {}".format("{}{}".format(directory, "{0:03}".format(i)), avg))
        file.write("{}, ".format(avg))
        file.flush()

In [ ]:
# Read data back from flat file and process
data = []
with open("natural_avgs.txt", "r") as file:
    txt = file.read()
    txt = txt.strip()
    txt = txt.split(",")
    txt = txt[:-1]

total = 0
ctr = 0

for each in txt:
    total += float(each)
    ctr += 1

avg = total/ctr
print(avg)


The natural average of all audio clips in the data-set combined is: 0.7374216270845926

This implies that the distrobution between 0's and 1's is not 50/50 and rather 27/73.

In [3]:
from threading import Thread

class ThreadWithReturnValue(Thread):
    
    def __init__(self, group=None, target=None, name=None,
                 args=(), kwargs={}, Verbose=None):
        Thread.__init__(self, group, target, name, args, kwargs)
        self._return = None

    def run(self):
        if self._target is not None:
            self._return = self._target(*self._args,
                                                **self._kwargs)
    def join(self, *args):
        Thread.join(self, *args)
        return self._return


In [ ]:
# Gets average bit value in each subdirectory of the data-set and writes it to a flat file
# Functionally same as above, this is threaded to save time

threads = [None]*156
results = [None]*156

for i in range(156):
    threads[i] = ThreadWithReturnValue(target=process_dir, args=("{}{}".format(directory, "{0:03}".format(i)), get_avg_echo_vals), name=str(i))
    print("Starting", threads[i])
    threads[i].start()

with open('natural_avgs.txt', 'a') as file:
    for thread in threads:
        if thread is not None:
            val = thread.join()
            file.write("{},".format(str(val)))
            file.flush()